#  ****Scientific Image Forgery Detection****

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
from pathlib import Path
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Deep Learning imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score

In [ ]:
class Config:
    RANDOM_STATE = 42
    BATCH_SIZE = 4  # Reduced for stability
    IMG_SIZE = (256, 256)
    EPOCHS = 10
    LR = 1e-4
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
config = Config()
print(f"Device: {config.DEVICE}")

# **Data Exploration and EDA**

In [ ]:
# Data paths
DATA_DIR = Path('/kaggle/input/recodai-luc-scientific-image-forgery-detection')
TRAIN_IMAGE_DIR = DATA_DIR / 'train_images'
TEST_IMAGE_DIR = DATA_DIR / 'test_images'
TRAIN_MASK_DIR = DATA_DIR / 'train_masks'


In [ ]:
# Find all image files
def find_all_image_files(directory):
    image_extensions = ['*.png', '*.jpg', '*.jpeg']
    images = []
    
    if not directory.exists():
        return images
    
    for ext in image_extensions:
        images.extend(directory.rglob(ext))
    
    return sorted(images)

train_images = find_all_image_files(TRAIN_IMAGE_DIR)
test_images = find_all_image_files(TEST_IMAGE_DIR)
train_masks = list(TRAIN_MASK_DIR.glob('*.npy')) if TRAIN_MASK_DIR.exists() else []

print(f"Training images: {len(train_images)}")
print(f"Test images: {len(test_images)}")
print(f"Training masks: {len(train_masks)}")

In [ ]:
# Create dataset mapping
def create_dataset_mapping():
    image_mask_pairs = []
    
    mask_ids = {mask_path.stem for mask_path in train_masks}
    
    for image_path in train_images:
        image_id = image_path.stem
        mask_path = TRAIN_MASK_DIR / f"{image_id}.npy"
        
        has_forgery = mask_path.exists()
        
        image_mask_pairs.append({
            'image_id': image_id,
            'image_path': image_path,
            'mask_path': mask_path if has_forgery else None,
            'has_forgery': has_forgery
        })
    
    return pd.DataFrame(image_mask_pairs)

df = create_dataset_mapping()
print(f"Total samples: {len(df)}")
print(f"Forged images: {df['has_forgery'].sum()}")
print(f"Authentic images: {(~df['has_forgery']).sum()}")

In [ ]:
def visualize_samples(df, num_samples=4):
    forged_samples = df[df['has_forgery'] == True].sample(min(num_samples, len(df[df['has_forgery'] == True])))
    
    fig, axes = plt.subplots(2, len(forged_samples), figsize=(15, 6))
    
    if len(forged_samples) == 1:
        axes = axes.reshape(2, 1)
    
    for idx, (_, sample) in enumerate(forged_samples.iterrows()):
        try:
            image = cv2.imread(str(sample['image_path']))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            mask = np.load(sample['mask_path'])
            if mask.ndim == 3 and mask.shape[0] == 1:
                mask = mask[0]
            
            axes[0, idx].imshow(image)
            axes[0, idx].set_title(f"Image: {sample['image_id']}")
            axes[0, idx].axis('off')
            
            axes[1, idx].imshow(mask, cmap='hot')
            axes[1, idx].set_title("Forgery Mask")
            axes[1, idx].axis('off')
            
        except Exception as e:
            print(f"Error visualizing {sample['image_id']}: {e}")
    
    plt.tight_layout()
    plt.show()

visualize_samples(df)

In [ ]:
# Fixed Dataset Class with proper mask handling
class ForgeryDataset(Dataset):
    def __init__(self, df, is_train=True):
        self.df = df.reset_index(drop=True)
        self.is_train = is_train
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        sample = self.df.iloc[idx]
        
        try:
            # Load image
            image = cv2.imread(str(sample['image_path']))
            if image is None:
                # Create consistent dummy image
                image = np.ones((512, 512, 3), dtype=np.uint8) * 128
            else:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Initialize mask
            mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.float32)
            
            # Load mask if available
            if self.is_train and sample['has_forgery'] and sample['mask_path'] is not None:
                try:
                    mask_data = np.load(sample['mask_path']).astype(np.float32)
                    
                    # Fix mask shape to always be 2D
                    if mask_data.ndim == 3:
                        if mask_data.shape[0] == 1:  # (1, H, W)
                            mask_data = mask_data[0]
                        elif mask_data.shape[2] == 1:  # (H, W, 1)
                            mask_data = mask_data[:, :, 0]
                        else:
                            # Take first channel if multiple channels
                            mask_data = mask_data[:, :, 0]
                    
                    # Ensure mask matches image dimensions
                    if mask_data.shape[:2] == image.shape[:2]:
                        mask = mask_data
                    else:
                        # Resize mask to match image if dimensions don't match
                        mask = cv2.resize(mask_data, (image.shape[1], image.shape[0]))
                        
                except Exception as e:
                    print(f"Mask loading error for {sample['image_id']}: {e}")
                    mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.float32)
            
            # Resize both to consistent size
            image_resized = cv2.resize(image, config.IMG_SIZE)
            mask_resized = cv2.resize(mask, config.IMG_SIZE)
            
            # Convert to tensor
            image_tensor = torch.from_numpy(image_resized).permute(2, 0, 1).float() / 255.0
            mask_tensor = torch.from_numpy(mask_resized).unsqueeze(0).float()
            
            return image_tensor, mask_tensor, sample['image_id']
            
        except Exception as e:
            # Return consistent dummy data on error
            print(f"Error loading {sample['image_id']}: {e}")
            image_tensor = torch.ones((3, *config.IMG_SIZE)).float() / 255.0
            mask_tensor = torch.zeros((1, *config.IMG_SIZE)).float()
            return image_tensor, mask_tensor, sample['image_id']

# **Prepare Data and Start Training**

In [ ]:
# Prepare data for training
train_df, val_df = train_test_split(
    df, 
    test_size=0.2, 
    random_state=config.RANDOM_STATE,
    stratify=df['has_forgery']
)

print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")

train_dataset = ForgeryDataset(train_df, is_train=True)
val_dataset = ForgeryDataset(val_df, is_train=True)

train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=0)

# Test one batch to ensure it works
print("Testing data loader...")
for images, masks, _ in train_loader:
    print(f"Image batch shape: {images.shape}")
    print(f"Mask batch shape: {masks.shape}")
    break


In [ ]:
# U-Net Model
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class UNet(nn.Module):
    def __init__(self, n_channels=3, n_classes=1):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(64, 128))
        self.down2 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(128, 256))
        self.down3 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(256, 512))
        
        self.up1 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.conv1 = DoubleConv(512, 256)
        self.up2 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.conv2 = DoubleConv(256, 128)
        self.up3 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.conv3 = DoubleConv(128, 64)
        
        self.outc = nn.Conv2d(64, n_classes, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        
        x = self.up1(x4)
        x = torch.cat([x, x3], dim=1)
        x = self.conv1(x)
        
        x = self.up2(x)
        x = torch.cat([x, x2], dim=1)
        x = self.conv2(x)
        
        x = self.up3(x)
        x = torch.cat([x, x1], dim=1)
        x = self.conv3(x)
        
        logits = self.outc(x)
        return self.sigmoid(logits)

model = UNet(n_channels=3, n_classes=1).to(config.DEVICE)
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Loss function
class DiceBCELoss(nn.Module):
    def __init__(self):
        super(DiceBCELoss, self).__init__()

    def forward(self, pred, target):
        smooth = 1.0
        pred_flat = pred.view(-1)
        target_flat = target.view(-1)
        
        intersection = (pred_flat * target_flat).sum()
        dice_loss = 1 - (2. * intersection + smooth) / (pred_flat.sum() + target_flat.sum() + smooth)
        bce_loss = nn.BCELoss()(pred_flat, target_flat)
        
        return dice_loss + bce_loss

criterion = DiceBCELoss()
optimizer = optim.Adam(model.parameters(), lr=config.LR)

In [ ]:
# Fixed Training function with error handling
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs):
    train_losses = []
    best_loss = float('inf')
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        processed_batches = 0
        
        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}')
        for batch_idx, (images, masks, _) in enumerate(pbar):
            try:
                # Skip if batch is empty
                if images.shape[0] == 0:
                    continue
                    
                images = images.to(config.DEVICE)
                masks = masks.to(config.DEVICE)
                
                # Skip if shapes don't match
                if images.shape[2:] != masks.shape[2:]:
                    continue
                
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, masks)
                loss.backward()
                optimizer.step()
                
                train_loss += loss.item()
                processed_batches += 1
                pbar.set_postfix({'Loss': f'{loss.item():.4f}'})
                
            except Exception as e:
                print(f"Error in batch {batch_idx}: {e}")
                continue
        
        if processed_batches == 0:
            print(f"Epoch {epoch+1}: No batches processed, skipping...")
            continue
            
        avg_train_loss = train_loss / processed_batches
        train_losses.append(avg_train_loss)
        
        print(f'Epoch {epoch+1}: Train Loss: {avg_train_loss:.4f}')
        
        if avg_train_loss < best_loss:
            best_loss = avg_train_loss
            torch.save(model.state_dict(), 'best_model.pth')
            print(f'New best model saved! Loss: {best_loss:.4f}')
    
    return train_losses

print("Starting training...")
train_losses = train_model(model, train_loader, val_loader, criterion, optimizer, config.EPOCHS)


In [ ]:
# Plot training loss
if train_losses:
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training Loss')
    plt.show()
else:
    print("No training data to plot")

In [ ]:
# RLE Encoding function
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    
    if len(runs) == 0:
        return "[]"
    else:
        return "[" + " ".join(str(x) for x in runs) + "]"

# **Submission**

In [ ]:
def create_final_submission():
    """Create submission with exact format: case_id,annotation"""
    
    # Initialize model (use trained if available, otherwise create new)
    try:
        model.load_state_dict(torch.load('best_model.pth'))
        print("Using trained model for predictions")
    except:
        print("Using default model for predictions")
        model = UNet(n_channels=3, n_classes=1).to(config.DEVICE)
    
    model.eval()
    predictions = []
    
    for test_img_path in tqdm(test_images, desc="Creating predictions"):
        case_id = test_img_path.stem
        
        try:
            # Load and process image
            image = cv2.imread(str(test_img_path))
            if image is None:
                predictions.append({'case_id': case_id, 'annotation': 'authentic'})
                continue
                
            original_shape = image.shape[:2]
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Resize for model
            image_resized = cv2.resize(image, config.IMG_SIZE)
            image_tensor = torch.from_numpy(image_resized).permute(2, 0, 1).float().unsqueeze(0) / 255.0
            image_tensor = image_tensor.to(config.DEVICE)
            
            # Predict
            with torch.no_grad():
                pred = model(image_tensor)
                pred_mask = pred.squeeze().cpu().numpy()
            
            # Process prediction
            pred_mask_resized = cv2.resize(pred_mask, (original_shape[1], original_shape[0]))
            binary_mask = (pred_mask_resized > 0.3).astype(np.uint8)  # Lower threshold for sensitivity
            
            # Decide authentic vs forgery
            if np.sum(binary_mask) < 50:  # Small area threshold
                annotation = 'authentic'
            else:
                annotation = rle_encode(binary_mask)
                
        except:
            annotation = 'authentic'
        
        predictions.append({'case_id': case_id, 'annotation': annotation})
    
    # Create DataFrame with exact column order
    submission_df = pd.DataFrame(predictions)[['case_id', 'annotation']]
    
    # Ensure proper sorting
    try:
        submission_df = submission_df.sort_values('case_id', key=lambda x: x.astype(int))
    except:
        submission_df = submission_df.sort_values('case_id')
    
    return submission_df

# Create and save final submission
final_submission = create_final_submission()
final_submission.to_csv('submission.csv', index=False)




In [ ]:
final_submission.head(10)